# 逻辑斯蒂回归

## 使用MNIST数据集中的0，1数据完成logistic regression 任务

### 导入相关包

In [32]:
import torch
import torchvision as tv
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

In [33]:
# 定义是否使用GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 定义数据预处理方式
transform = transforms.ToTensor()

利用框架自带数据集读取方式读取 MNIST数据集

In [34]:
# download 第一次可设置为Ture
trainset = tv.datasets.MNIST(
    root='./data/',
    train=True,
    download=True,
    transform=transform)

从训练集中取出0、1标签的数据构成此次任务的训练集(train_input,train_label) 

In [35]:
train_input = []
train_label = []
for data, label in trainset:
    if label == 0 or label == 1:
        train_input.append(data)
        train_label.append(label)

In [36]:
print(len(train_input), len(train_input[0]), len(train_input[0][0][0]))

12665 1 28


从测试集中取出0、1标签的数据构成此次任务的测试集(test_input,test_lable)

In [37]:
# 定义测试数据集
testset = tv.datasets.MNIST(
    root='./data/',
    train=False,
    download=True,
    transform=transform)

test_input = []
test_label = []
for data, label in testset:
    if label == 0 or label == 1:
        test_input.append(data)
        test_label.append(label)
# test_input = torch.stack(test_input).to(device)
# test_label = torch.Tensor(test_label).int()

 创建自己的数据集，需要补充完成

In [38]:
class TensorDataset(torch.utils.data.Dataset):
    """
    创建自己的dataset类 继承torch.utils.data.Dataset。
    Dataset wrapping data and target tensors.
    Each sample will be retrieved by indexing both tensors along the first
    dimension.
    Arguments:
        data_tensor (Tensor): contains sample data.
        target_tensor (Tensor): contains sample targets (labels).
    """
    def __init__(self, data_tensor, target_tensor):
        self.data_tensor = data_tensor
        self.target_tensor = target_tensor

    # 这个方法是必须要有的，用于按照索引读取每个元素的具体内容
    def __getitem__(self, index):
        # #return很关键，return回哪些内容，那么我们在训练时循环读取每个batch时，
        # 就能获得哪些内容
        return self.data_tensor[index],self.target_tensor[index]


    ##这个函数也必须要写，它返回的是数据集的长度
    def __len__(self):
        return len(self.data_tensor)

定义训练批处理数据

In [39]:
# batch_size 根据自己计算资源设计，shuffle设置为True，查询设置的原因。
#train_dataset = TensorDataset(train_input, train_label),
trainloader = torch.utils.data.DataLoader(
    TensorDataset(train_input, train_label),
    batch_size=128,
    shuffle=True,
    )
test_dataset = TensorDataset(testset.data, testset.targets)
testloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=128,
    shuffle=False  # 在测试时不需要 shuffle
)

利用pytorch框架的nn.Module 模块完成logistic regression 模型的设计

In [40]:
class logistic_regression(nn.Module):
    '''
        逻辑回归模型
    '''
    def __init__(self):
        super(logistic_regression, self).__init__()
        self.linear = nn.Linear(784, 1)

    def forward(self, x):
        x = x.view(x.shape[0], -1)  # 将图像展平为一维向量
        out = self.linear(x)
        return out

In [41]:
model = logistic_regression()

设置损失函数及优化器。提示：使用binary cross entropy 损失函数，优化器可设置SGD、Adam等

In [42]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [43]:
test_input = torch.stack(test_input)
test_label = torch.Tensor(test_label).int()

训练

In [44]:


for epoch in range(5):
    sum_loss = 0.0
    # 数据读取
    for i, data in enumerate(trainloader):
        inputs, labels = data
        inputs, labels = inputs, labels.view(-1, 1)
        # 梯度清零
        optimizer.zero_grad()
        # 模型输出
        pred = model(inputs)
        # 定义loss
        loss = criterion(pred, labels.float())
        # loss反向传播
        loss.backward()
        # 更新参数
        optimizer.step()
        
        sum_loss += loss.item()

    print('第{0}轮 loss: {1}'.format(epoch+1, sum_loss / 100))

#  输出测试集准确率: 没有对测试集进行DataLoader，如果自己电脑配置不够，
#尝试建立testloader 然后按照批量计算
    with torch.no_grad():
        test_outputs = model(test_input)

        test_predicted = torch.squeeze(test_outputs>=0.5).int()
        
        accuracy = (test_predicted.cpu() == test_label.int()).sum().float() / test_label.size()[0]
        print('识别准确率为：{}%'.format(accuracy*100))

第1轮 loss: 0.2653926795721054
识别准确率为：99.81087493896484%
第2轮 loss: 0.09179407887160779
识别准确率为：99.90543365478516%
第3轮 loss: 0.06078353103250265
识别准确率为：99.90543365478516%
第4轮 loss: 0.047019064985215665
识别准确率为：99.90543365478516%
第5轮 loss: 0.03905912417918444
识别准确率为：99.90543365478516%


### 附加
#### 尝试将模型输出变为[p(y=0|x),p(y=1|x)] 的向量，观察参数（和正常logistic模型）的变化。
#### 提示：损失函数由BCE变为CE

In [45]:
class logistic_regression(nn.Module):
    '''
        逻辑回归模型
    '''
    def __init__(self):
        super(logistic_regression, self).__init__()
        # 添加logistic regression层
        # 注意：如果你的输入数据的维度不是1，你需要修改这里的输入维度
        self.linear = torch.nn.Linear(784, 2)  # 输出两个值

    def forward(self, x):
        # 计算图（1）数据维度拉长
        x = x.view(x.shape[0], -1)
        # （2）logistic regression层传播
        y_pred = self.linear(x)
        # 使用softmax函数
        y_pred = torch.nn.functional.softmax(y_pred, dim=1)
        # （3）return输出（注意维度）
        return y_pred
# 设置损失函数及优化器
criterion = nn.CrossEntropyLoss()  # 使用交叉熵损失
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
for epoch in range(5):
    sum_loss = 0.0
    # 数据读取
    for i, data in enumerate(trainloader):
        inputs, labels = data
        inputs, labels = inputs, labels.view(-1, 1)  # 调整标签的维度
        # 梯度清零
        optimizer.zero_grad()

        # 模型输出
        outputs = model(inputs)
        # 定义loss
        loss = criterion(outputs, labels.float())
        # loss反向传播
        loss.backward()
        # 更新参数
        optimizer.step()

        sum_loss += loss.item()

    print('第{0}轮 loss: {1}'.format(epoch+1, sum_loss / 100))

#  输出测试集准确率: 没有对测试集进行DataLoader，如果自己电脑配置不够，
#尝试建立testloader 然后按照批量计算
    with torch.no_grad():
        test_outputs = model(test_input.cpu())

        test_predicted = torch.squeeze(test_outputs>=0.5).int()

        accuracy = (test_predicted.cpu() == test_label.int()).sum().float() / test_label.size()[0]
        print('识别准确率为：{}%'.format(accuracy*100))
# for name, param in model.named_parameters():
#     print(name, param.data)

第1轮 loss: 0.0
识别准确率为：99.90543365478516%
第2轮 loss: 0.0
识别准确率为：99.90543365478516%
第3轮 loss: 0.0
识别准确率为：99.90543365478516%
第4轮 loss: 0.0
识别准确率为：99.90543365478516%
第5轮 loss: 0.0
识别准确率为：99.90543365478516%
